In [1]:
import datasets
import os
import re

os.environ['https_proxy'] = '127.0.0.1:1080'
os.environ['http_proxy'] = '127.0.0.1:1080'
CACHE_DIR = "/home/liuchao/shushanfu/LMOps/data/huggingface_cache"

# only get one packet
dataset = datasets.load_dataset('bigcode/starcoderdata', split='train',data_files="python/train-00000-of-00059.parquet",trust_remote_code=True,cache_dir=CACHE_DIR)


In [2]:
dataset

Dataset({
    features: ['max_stars_repo_path', 'max_stars_repo_name', 'max_stars_count', 'id', 'content'],
    num_rows: 218079
})

In [26]:
print(dataset[2019]["content"])

<reponame>SDhuangao/netease-cloud-music-dl
# -*- coding: utf-8 -*-

import requests

from ncm.encrypt import encrypted_request
from ncm.constants import headers
from ncm.constants import song_download_url
from ncm.constants import get_song_url
from ncm.constants import get_album_url
from ncm.constants import get_artist_url
from ncm.constants import get_playlist_url


class CloudApi(object):

    def __init__(self, timeout=30):
        super().__init__()
        self.session = requests.session()
        self.session.headers.update(headers)
        self.timeout = timeout

    def get_request(self, url):

        response = self.session.get(url, timeout=self.timeout)
        result = response.json()
        if result['code'] != 200:
            print('Return {} when try to get {}'.format(result, url))
        else:
            return result

    def post_request(self, url, params):

        data = encrypted_request(params)
        response = self.session.post(url, data=data, timeout=self.

In [19]:
wordCount = 0
newline_count = 0  # 回车 \n
space_count = 0    # 空格
tab_count = 0      # 制表符 \t
for data in dataset:
    content = data['content']
    # 拆分成单词并更新计数
    wordCount += len(content.split())
    newline_count += content.count('\n')
    space_count += content.count(' ')
    tab_count += content.count('\t')
print(f"wordCount: {wordCount}")
print(f"Newlines: {newline_count}")
print(f"Spaces: {space_count}")
print(f"Tabs: {tab_count}")
allCount = wordCount+newline_count+space_count+tab_count

wordCount: 88962456
Newlines: 28874698
Spaces: 249641192
Tabs: 1872325


In [29]:
(wordCount+newline_count+space_count+tab_count)/dataset.num_rows

1693.6553771798294

In [28]:
wordCount+newline_count+tab_count

119709479

In [22]:
allCount

369350671

In [25]:
average = allCount/dataset.num_rows
print(average)

548.9271273254187


In [18]:
dataset[218000]["content"].count('\t')

15

In [1]:
import torch

# 定义一个模拟的相似度计算类
class SimilarityComputer:
    def compute_similarity(self, q_reps, p_reps):
        if len(p_reps.size()) == 2:
            return torch.matmul(q_reps, p_reps.transpose(0, 1))
        return torch.matmul(q_reps, p_reps.transpose(-2, -1))

# 实例化这个类
sim_computer = SimilarityComputer()

# 创建一些示例数据
# 假设有3个查询和4个页面/文档，每个表示的维度是5
q_reps = torch.randn(3, 5)
p_reps = torch.randn(4, 5)

# 计算相似性
similarity_matrix = sim_computer.compute_similarity(q_reps, p_reps)

# 打印相似性矩阵
similarity_matrix


tensor([[-2.1845e+00,  4.5742e+00,  3.0843e+00, -4.4525e-01],
        [ 5.6453e-01,  1.2282e+00,  7.9614e-01,  1.9521e+00],
        [ 8.9111e-01, -7.5050e-01,  3.8116e-04, -3.5875e-01]])

In [2]:
q_reps

tensor([[ 0.7618,  0.5248, -1.4861, -1.7878, -0.3803],
        [ 1.3576,  0.3313,  0.5518, -0.7317,  0.1051],
        [ 1.0891, -0.6134, -0.0453,  0.6692, -0.1062]])

In [3]:
p_reps

tensor([[ 0.2450, -1.2239,  1.0084, -0.0141,  0.6720],
        [ 0.1995,  0.1531, -0.1556, -1.7364, -2.6459],
        [ 0.2211, -1.2953, -0.4167, -1.6134, -0.2422],
        [ 0.6901,  2.3909,  1.1986,  0.4577, -0.9828]])

In [7]:
similarity_matrix.size(1)

4

In [12]:
torch.arange(3) * 1

tensor([0, 1, 2])

In [11]:
4//3

1

In [3]:
import torch
def dense_score(self, q_reps, p_reps):
    scores = self.compute_similarity(q_reps, p_reps) / self.temperature
    scores = scores.view(q_reps.size(0), -1)
    return scores

def sparse_score(self, q_reps, p_reps):
    scores = self.compute_similarity(q_reps, p_reps) / self.temperature
    scores = scores.view(q_reps.size(0), -1)
    return scores

def colbert_score(self, q_reps, p_reps, q_mask: torch.Tensor):
    token_scores = torch.einsum('qin,pjn->qipj', q_reps, p_reps)
    scores, _ = token_scores.max(-1)
    scores = scores.sum(1) / q_mask[:, 1:].sum(-1, keepdim=True)
    scores = scores / self.temperature
    return scores

def compute_loss(self, scores, target):
    return self.cross_entropy(scores, target)

In [ ]:

idxs = torch.arange(q_dense_vecs.size(0), device=q_dense_vecs.device, dtype=torch.long)
targets = idxs * (p_sparse_vecs.size(0) // q_sparse_vecs.size(0))
dense_scores = dense_score(q_dense_vecs, p_dense_vecs)  # B, B * N

loss = compute_loss(dense_scores, targets)

In [12]:
import torch
import torch.nn.functional as F

def cosine_similarity_loss(transformer_features, target_features):
    # 计算余弦相似度
    similarity = F.cosine_similarity(transformer_features, target_features, dim=-1)
    
    # 将相似度转换为损失，注意这里我们直接计算1 - similarity作为损失
    loss = 1 - similarity
    return loss.mean()  # 对所有样本的损失求平均

# 创建示例特征向量
transformer_features = torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]], dtype=torch.float32)
target_features = torch.tensor([[0.2, 0.2, 0.2], [0.5, 0.5, 0.5]], dtype=torch.float32)

# 计算损失
loss = cosine_similarity_loss(transformer_features, target_features)
print("Cosine Similarity Loss:", loss.item())


Cosine Similarity Loss: 0.043626248836517334


In [13]:
# 创建示例特征向量
transformer_features = torch.tensor([[0.1, 0.1, 0.3], [0.4, 0.5, 0.4]], dtype=torch.float32)
target_features = torch.tensor([[0.9, 0.9, 0.1], [0.6, 0.5, 0.6]], dtype=torch.float32)

# 计算损失
loss = cosine_similarity_loss(transformer_features, target_features)
print("Cosine Similarity Loss:", loss.item())

Cosine Similarity Loss: 0.26115649938583374


In [14]:
# 创建示例特征向量
transformer_features = torch.tensor([[0.1, 0.1, 0.3], [0.4, 0.5, 0.4]], dtype=torch.float32)
target_features = torch.tensor([[0.3, 0.4, 0.5], [0.6, 0.7, 0.8]], dtype=torch.float32)

# 计算损失
loss = cosine_similarity_loss(transformer_features, target_features)
print("Cosine Similarity Loss:", loss.item())

Cosine Similarity Loss: 0.03723827004432678


In [10]:
import datasets

# 加载数据集
CACHE_DIR = "/home/liuchao/shushanfu/LMOps/data/huggingface_cache"

# only get one packet
dataset = datasets.load_dataset('bigcode/starcoderdata', split='train',data_files="python/train-00000-of-00059.parquet",trust_remote_code=True,cache_dir=CACHE_DIR)

# 初始化最小长度为一个较大的数
min_length = float('inf')
total_min_256 = 0
# 遍历数据集中的每个样本
for example in dataset:
    wordCount = 0
    newline_count = 0  # 回车 \n
    space_count = 0    # 空格
    tab_count = 0      # 制表符 \t
    content = example['content']
    # 拆分成单词并更新计数
    wordCount += len(content.split())
    newline_count += content.count('\n')
    space_count += content.count(' ')
    tab_count += content.count('\t')
    # 计算样本字符串长度，包括空格和制表符
    allCount = wordCount+newline_count+space_count+tab_count
    # 更新最小长度
    min_length = min(min_length, allCount)
    if min_length<256:
        total_min_256+=1

# 输出最小长度
print("最短的字符长度（包括空格和制表符）为:", min_length)


最短的字符长度（包括空格和制表符）为: 1


In [11]:
total_min_256

218078

In [5]:
len("1,2,0.0.2564546546541432123  45634123")

37